## Setup

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-235194
Azure region: westeurope
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-235194


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "eduProject1"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    cluster = ComputeTarget(ws, cluster_name)
    print("Found existing cluster!")
except ComputeTargetException:
    print("No cluster found, creating one...")
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", min_nodes=0, max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion(show_output=True)

compute_target = ws.compute_targets[cluster_name]

No cluster found, creating one...
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## HyperDrive

In [11]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "C": uniform(0.1, 2.0),
    "max_iter": choice(10, 20, 50, 100, 200)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=5, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
# Using SKLearn because ScriptRunConfig doesn't seem to work
# src = ### YOUR CODE HERE ###
estimator = SKLearn(source_directory=".", compute_target=compute_target, entry_script="train.py")

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
    estimator=estimator,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=5
)

In [12]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_7080e9fb-0533-4e65-bdd3-749c6440fd33
Web View: https://ml.azure.com/runs/HD_7080e9fb-0533-4e65-bdd3-749c6440fd33?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-235194/workspaces/quick-starts-ws-235194&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-06-06T19:50:33.448760][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space
[2023-06-06T19:50:34.0286162Z][SCHEDULER][INFO]Scheduling job, id='HD_7080e9fb-0533-4e65-bdd3-749c6440fd33_1' 
[2023-06-06T19:50:33.9380473Z][SCHEDULER][INFO]Scheduling job, id='HD_7080e9fb-0533-4e65-bdd3-749c6440fd33_0' 
[2023-06-06T19:50:34.1187618Z][SCHEDULER][INFO]Scheduling job, id='HD_7080e9fb-0533-4e65-bdd3-749c6440fd33_2' 
[2023-06-06T19:50:34.2396340Z][SCHEDULER][INFO]Scheduling job, id='HD_7080e9fb-0533-4e65-bdd3-749c6440fd33_3' 
[2023-06-06T19:50:34.2809964Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_7080e9fb-0533-4e65-bdd3-749c6440fd33_0

{'runId': 'HD_7080e9fb-0533-4e65-bdd3-749c6440fd33',
 'target': 'eduProject1',
 'status': 'Completed',
 'startTimeUtc': '2023-06-06T19:50:32.851859Z',
 'endTimeUtc': '2023-06-06T19:52:10.718327Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '870caf2b-65eb-4e3b-806e-64b054ae1bf1',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1035-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.49.0',
  'space_size': 'infinite_space_size',
  'score': '0.9113862588006798',
  'best_child_run_id': 'HD_7080e9fb-0533-4e65-bdd3-749c6440fd33_3',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_7080e9fb-0533-4e65-bdd3-749c6440fd33_3'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValu

KeyError: 'log_files'

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [13]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print(f"Best run id: {best_run.id}")
print(f"Best run accuracy: {best_run_metrics['Accuracy']}")

Best run id: HD_7080e9fb-0533-4e65-bdd3-749c6440fd33_3
Best run accuracy: 0.9113862588006798


In [14]:
best_run.get_file_names()

['outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [15]:
best_run.download_files()

In [16]:
model = best_run.register_model(
    model_name="model.joblib",
    model_path="outputs/model.joblib"
)

## AutoML

In [17]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset = TabularDatasetFactory.from_delimited_files(path=data_url)

In [18]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)

In [19]:
# Add target to x dataframe
x["y"] = y

In [20]:
datastore = ws.get_default_datastore()
train_dataset = TabularDatasetFactory.register_pandas_dataframe(
    dataframe=x,
    target=datastore,
    name="train_dataset"
)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/5b73cfab-12c3-4ec3-95f3-c1cbfb5c91af/
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'nr.employed' -> 'nr_employed'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'job_admin.' -> 'job_admin_'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'education_basic.4y'

In [21]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_dataset,
    label_column_name="y",
    n_cross_validations=5,
    compute_target=compute_target
)

In [22]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_486dd7d5-aca0-409e-84b8-d87a7ec36f63,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_486dd7d5-aca0-409e-84b8-d87a7ec36f63,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+-----

{'runId': 'AutoML_486dd7d5-aca0-409e-84b8-d87a7ec36f63',
 'target': 'eduProject1',
 'status': 'Completed',
 'startTimeUtc': '2023-06-06T20:02:03.482668Z',
 'endTimeUtc': '2023-06-06T20:45:45.860622Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'eduProject1',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30","resource_group":"aml-quickstarts-235194","workspace_name":"quick-starts-ws-235194","region":"westeurope","compute_target":"eduProject1","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enab

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Del

In [23]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run, best_automl_model = automl_run.get_output()

Package:azureml-automl-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-core, training version:1.50.0, current version:1.49.0
Package:azureml-dataprep, training version:4.10.6, current version:4.9.1
Package:azureml-dataprep-rslex, training version:2.17.5, current version:2.16.1
Package:azureml-dataset-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-defaults, training version:1.50.0, current version:1.49.0
Package:azureml-interpret, training version:1.50.0, current version:1.49.0
Package:azureml-mlflow, training version:1.50.0, current version:1.49.0
Package:azureml-pipeline-core, training version:1.50.0.post1, current version:1.49.0
Package:azureml-responsibleai, training version:1.50.0, current version:1.49.0
Package:azureml-telemetry, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-client, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-runtime, training version:1.50.0, current 

In [24]:
best_automl_run.register_model(
    model_name="model.pkl",
    model_path="./outputs"
)

Model(workspace=Workspace.create(name='quick-starts-ws-235194', subscription_id='61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30', resource_group='aml-quickstarts-235194'), name=model.pkl, id=model.pkl:1, version=1, tags={}, properties={})

In [25]:
# Clean up
compute_target.delete()